In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sbe_vallib import Validation, BinaryScorer, SupervisedSampler

## Dataset

In [3]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

X, y = make_classification(n_samples=120, n_classes=2,
                               n_features=5, n_informative=3, n_redundant=0,
                               random_state=0)

X = pd.DataFrame(X)
y = pd.Series(y)

X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, stratify=y_train)


In [4]:
len(X_train), len(X_test), len(X_val)

(75, 12, 33)

## Model

In [5]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(X_train, y_train)

## Sampler Scorer

In [6]:
from sbe_vallib import SupervisedSampler, BinaryScorer

sampler = SupervisedSampler(train={'X': X_train, 'y_true': y_train, 'y_pred': model.predict_proba(X_train)},
                            oos={'X': X_test, 'y_true': y_test, 'y_pred': model.predict_proba(X_test)},
                            oot={'X': X_val, 'y_true': y_val, 'y_pred': model.predict_proba(X_val)})

scorer = BinaryScorer()

In [32]:
from sbe_vallib.table.model_quality.test_ci import test_ci

def custom_test(model, scorer, sampler, threshold = 1, **kwargs):
    semaphore = 'green'
    if len(sampler.train['X']) < threshold:
        semaphore = 'red'
    
    df_table = pd.DataFrame({'len': [len(sampler.train['X'])],
                             'semaphore': semaphore})
    
    return {
        "semaphore": semaphore,
        "result_dict": {'train_shape':sampler.train['X'].values.shape},
        "result_dataframes": [df_table],
        "result_plots": [],
    }


custom_tests = {
    "custom_1": {"block": "data_quality", "callable": custom_test, "params": {"threshold": 5000}},
    "custom_test_ci": {"block": "model_quality", "callable": test_ci, "params": {"gen_method": "bootstrap"}},
}



validor = Validation(model, sampler, scorer, custom_tests=custom_tests,
                         pipeline='../src/sbe_vallib/table/pipelines/Config_31.xlsx')
res = validor.validate(save_excel=True)
res.keys()

/Users/azatsultanov/Programming/vallib/vallib_env/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


dict_keys(['test_factor_psi', 'test_ci', 'test_key_metric', 'test_key_metric_stability', 'custom_1', 'custom_test_ci'])

In [9]:
pd.DataFrame([{'a': 'a', 'b': 5.2342342}]).round(3)

,a,b
0,a,5.234


In [24]:
validor.pipeline['tests_desc']['test_key_metric_stability']

{'Название': 'Тест\xa05.1 Анализ стабильности модели на выборках out-of-sample и out-of-time по Gini',
 'Цель': 'Оценить стабильность модели по ключевой метрике качества Gini.',
 'Интерпретация': 'Чем слабее падает коэффициент Gini, тем стабильнее модель во времени. Красный цвет означает, что с течением времени качество модели серьезно ухудшается, и модель не рекомендуется к эксплуатации. Желтый цвет говорит о граничном случае, когда следует обратить внимание на возможность значительного ухудшения модели и провести анализ коэффициента Gini через определенный временной интервал.',
 'Границы красный': 'Абсолютное снижение более 25\xa0п.\xa0п. и  относительное снижение более 30%',
 'Границы желтый': 'Абсолютное снижение более 15 п. п. И относительное снижение более 20%',
 'Границы зеленый': 'Абсолютное снижение менее 15 п. п. ИЛИ относительное снижение менее 20%'}

In [ ]:
res

In [8]:
from sbe_vallib.xlsx_aggregator import XlsxBot

In [1]:
from io import BytesIO
from urllib.request import urlopen


import xlsxwriter

# Create the workbook and add a worksheet.
workbook  = xlsxwriter.Workbook('images_bytesio.xlsx')
worksheet = workbook.add_worksheet()


In [12]:
import io
from PIL import Image


def PIL2IOBytes(pil_image):
    buf = io.BytesIO()
    pil_image.save(buf, format='JPEG')
    return buf

image_data = PIL2IOBytes(res['test_ci']['result_plots'][0])

# Write the byte stream image to a cell. Note, the filename must be
# specified. In this case it will be read from url string.
worksheet.insert_image('B2', url, {'image_data': image_data})


workbook.close()

In [13]:
PIL2IOBytes(res['test_ci']['result_plots'][0])